In [1]:
!pwd

/home/jovyan/work/unit1


In [44]:
!wget \
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet \
-P /home/jovyan/work/data/yellow

--2024-05-16 16:56:07--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... ::ffff:52.85.221.164, ::ffff:52.85.221.138, ::ffff:52.85.221.106, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|::ffff:52.85.221.164|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘/home/jovyan/work/data/yellow/yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  2.95MB/s    in 16s     

2024-05-16 16:56:25 (2.81 MB/s) - ‘/home/jovyan/work/data/yellow/yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [45]:
!wget \
https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet \
-P /home/jovyan/work/data/yellow

--2024-05-16 16:56:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... ::ffff:52.85.221.138, ::ffff:52.85.221.106, ::ffff:52.85.221.164, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|::ffff:52.85.221.138|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘/home/jovyan/work/data/yellow/yellow_tripdata_2023-02.parquet’

yellow_tripdata_202 100%[===================>]  45.54M  2.84MB/s    in 16s     

2024-05-16 16:56:43 (2.79 MB/s) - ‘/home/jovyan/work/data/yellow/yellow_tripdata_2023-02.parquet’ saved [47748012/47748012]



In [2]:
import os

import pandas as pd

from sklearn.feature_extraction import DictVectorizer

In [4]:
def read_dataframe(filename: str) -> pd.DataFrame:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    return df

def columns_number(df: pd.DataFrame) -> None:
    print(len(df.columns))

# Q1. Downloading the data

In [5]:
dir = "/home/jovyan/work/data/yellow/"
filename = "yellow_tripdata_2023-01.parquet"
filepath = os.path.join(dir, filename)

In [6]:
df = read_dataframe(filepath)
no_columns = columns_number(df)

19


# Q2. Computing duration

In [5]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [7]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration'] = df.duration.dt.total_seconds() / 60

In [7]:
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [8]:
df.duration.std()

42.594351241920904

# Q3. Dropping outliers

In [2]:
df_train = df[(df.duration >= 1) & (df.duration <= 60)]

NameError: name 'df' is not defined

In [12]:
start_len = len(df)
end_len = len(df_train)

In [13]:
start_len, end_len

(3066766, 3009173)

In [23]:
end_len / start_len * 100

98.1220282212598

# Q4. 

In [3]:
def read_dataframe_remove_outliers(filename: str) -> pd.DataFrame:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [4]:
dir = "/home/jovyan/work/data/yellow/"
filename = "yellow_tripdata_2023-01.parquet"
filepath = os.path.join(dir, filename)

df_train = read_dataframe_remove_outliers(filepath)

In [5]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df_train[categorical] = df_train[categorical].astype(str)

train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [ ]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [ ]:
X_train.shape